# Chapter 08: Tupperware

This chapter unveils the foundations of control flow, error handling, state, and effects, extending the power of pure functions.<br><br>


---
<br>

#### The Mighty Container:

In [1]:
class Container:
    def __init__(self, x):
        self.value = x
    
    @staticmethod
    def of(x):
        return Container(x)

In [2]:
print(Container.of(3).value)                                            # Container(3)

print(Container.of('hotdogs').value)                                    # Container("hotdogs")

print(Container.of(Container.of({ 'name': 'yoda' })).value.value)       # Container(Container({ name: 'yoda' }))

3
hotdogs
{'name': 'yoda'}


<u><b>Rules:</b></u>
- Container is an object with <u>one property</u>. <br>Lots of containers just hold <u>one thing</u>, though they aren't limited to one. <br>We've arbitrarily named its property <u>value</u>.

- The value <u>cannot be one specific type</u>.

- Once data goes <u>into</u> the Container it <u>stays there</u>. <br>We could get it out by using <u>.value</u>, but that would <u>defeat</u> the purpose.

---

<br>

#### My First Functor:

<b>Defintion</b><br>
In mathematics, specifically category theory, a functor is a mapping between categories.

We need a way to run functions on our container.

In [3]:
# (a -> b) -> Container a -> Container b
Container.map = lambda self, f: Container.of(f(self.value))

In [4]:
e1 = Container.of(2).map(lambda two: two + 2)
print(f"Expect: Container(4):\t\t\tOutput: {e1.value}")

e2 = Container.of('flamethrowers').map(lambda s: str.upper(s))
print(f"Expect: Container('FLAMETHROWERS'):\tOutput: {e2.value}")

e3 = Container.of('bombs').map(lambda s: f'{s} away').map(len)
print(f"Expect: Container(10):\t\t\tOutput: {e3.value}")

del e1, e2, e3

Expect: Container(4):			Output: 4
Expect: Container('FLAMETHROWERS'):	Output: FLAMETHROWERS
Expect: Container(10):			Output: 10


#### Schrödinger's Maybe :


In [5]:
from utils import curry, compose, map, trace
from functools import partial
import re

In [6]:
class Maybe:
    @staticmethod
    def of(x):
        return Maybe(x)

    def isNothing(self):
        return not self.value

    def __init__(self, x):
        self.value = x
  

    map = curry(lambda self, fn: self if self.isNothing() else Maybe.of(fn(self.value)))
  
    # The equivalent of inspect() in python is __str__ method
    def __str__(self):
        return 'Nothing' if self.isNothing() else f'Just({self.value})'

In [7]:
match = curry(lambda reg, s:  bool(re.findall(reg, s)))
prop = curry(lambda p, obj: obj[p] if p in obj else None)
add = curry(lambda n, x: n+x)

In [8]:
e1 = Maybe.of('Malkovich Malkovich').map(match(r'a'))
print(f"Expect: Just(True)\tOutput: {e1}")

e2 = Maybe.of(None).map(match(r'a'))
print(f"Expect: Nothing\t\tOutput: {e2}")

e3 = Maybe.of({ 'name': 'Boris' }).map(prop('age')).map(add(10))
print(f"Expect: Nothing\t\tOutput: {e3}")

e4 = Maybe.of({ 'name': 'Dinah', 'age': 14 }).map(prop('age')).map(add(10))
print(f"Expect: Just(24)\tOutput: {e4}")

del e1, e2, e3, e4

Expect: Just(True)	Output: Just(True)
Expect: Nothing		Output: Nothing
Expect: Nothing		Output: Nothing
Expect: Just(24)	Output: Just(24)


<b>Note for map:</b><br>
```map :: Functor f => (a -> b) -> f a -> f b```

---

#### Use Cases


In [9]:
# safeHead :: [a] -> Maybe(a)
safeHead = lambda xs : Maybe.of(xs[0] if xs else None)

# streetName :: Object -> Maybe String
streetName = compose(Maybe.map(fn=prop('street')), safeHead, prop('addresses'))

In [10]:
e1 = streetName({ 'addresses': [] })
print(f"Expect: Nothing\t\t\tOutput: {e1}")

e2 = streetName({ 'addresses': [{ 'street': 'Shady Ln.', 'number': 4201 }] })
print(f"Expect: Just('Shady Ln.')\tOutput: {e2}")

del e1, e2

Expect: Nothing			Output: Nothing
Expect: Just('Shady Ln.')	Output: Just(Shady Ln.)
